# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [12]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
# school_data_to_load = "Resources/schools_complete.csv"
weather_data = "../../WeatherPy/output_data/weather_file.csv"
weather_df = pd.read_csv(weather_data)
weather_df

,City,Country,City ID,Latitude,Longitude,Temperature,Max Temp,Humidity,Wind,Clouds,Date
0,Touros,BR,3386213,-5.20,-35.46,86.00,86.00,55,18.34,75,1602595870
1,Chālūs,IR,139223,36.66,51.42,69.80,69.80,60,6.93,0,1602595871
2,Portland,AU,2152667,-38.33,141.60,47.12,47.12,82,6.73,81,1602595874
3,Ozar,IN,1261068,20.10,73.93,77.40,77.40,75,8.23,100,1602595875
4,San Patricio,MX,3985168,19.22,-104.70,77.00,77.00,73,6.93,40,1602595877
...,...,...,...,...,...,...,...,...,...,...,...
495,Mossamedes,AO,3347019,-15.20,12.15,75.20,75.20,69,16.11,20,1602596510
496,Bismarck,US,5688025,46.81,-100.78,37.54,39.20,80,13.87,1,1602596372
497,Cho Dok,VN,1585660,10.70,105.12,76.69,76.69,92,13.98,100,1602596342
498,Layou,VC,3577879,13.20,-61.27,84.20,84.20,83,2.24,20,1602596513


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [14]:
# Configure gmaps
gmaps.configure(api_key=g_key)

In [15]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]].astype(float)
humid=weather_df["Humidity"].astype(float)

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humid, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
